In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
using Test
using ForwardDiff
using SpecialFunctions
using Polynomials
using BenchmarkTools
using TransportMap
using QuadGK
using FastGaussQuadrature
using AdaptiveTransportMap: derivative, vander

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1260


### Hermite functions

In [27]:
    # k = 3: check third derivative
    N = 20
    m = 10
    k = 3
    x = 0.2*randn(N)

    V = vander(m, k, x)

    @test size(V)==(N, m+1)

    for i=3:m
        @test norm(V[:,i+1] - (FamilyPhyHermite[i-2]).(x)*2*i*2*(i-1)*2*(i-2))<1e-6
        @test norm(V[:,i+1] - (FamilyPhyHermite[i-2]).(x)*2^k*factorial(i)/factorial(i-k))<1e-6

    end

In [23]:
    for i=0:m
        factor = 2^k*exp(loggamma(i+1) - loggamma(i+1-k))

        @test norm(V[:,i+1] - (FamilyScaledPhyHermite[i+1]).(x)*sqrt(factor))<1e-8
    end

Test Failed at In[23]:4
  Expression: norm(V[:, i + 1] - (FamilyScaledPhyHermite[i + 1]).(x) * sqrt(factor)) < 1.0e-8
   Evaluated: 1.1130004008792747 < 1.0e-8


Test.FallbackTestSetException: There was an error during testing

In [70]:
V

20×11 Array{Float64,2}:
 0.0  0.0  2.82843  -0.0464422  -13.8558  …  -1795.75     135.133  31870.0
 0.0  0.0  2.82843   2.22159    -12.4317     -1260.85   -5813.4    19430.4
 0.0  0.0  2.82843  -0.37977    -13.8148     -1779.82    1101.75   31493.2
 0.0  0.0  2.82843  -0.474233   -13.7915     -1770.79    1373.49   31279.8
 0.0  0.0  2.82843   0.828281   -13.6584     -1719.35   -2375.82   30066.4
 0.0  0.0  2.82843  -1.61258    -13.1057  …  -1509.35    4440.9    25154.0
 0.0  0.0  2.82843   0.474732   -13.7913     -1770.74   -1374.92   31278.5
 0.0  0.0  2.82843   1.58483    -13.1313     -1518.96   -4372.82   25377.3
 0.0  0.0  2.82843  -2.7998     -11.5935      -963.462   6859.62   12716.7
 0.0  0.0  2.82843   2.39007    -12.2074     -1180.01   -6146.64   17590.4
 0.0  0.0  2.82843   1.50292    -13.2044  …  -1546.41   -4169.41   26016.0
 0.0  0.0  2.82843  -3.1803     -10.9367      -739.291   7385.54    7760.69
 0.0  0.0  2.82843  -2.50019    -12.0519     -1124.52    6352.26   16333.7


In [59]:
FamilyPhyHermite[i].(x)*sqrt(2*exp(loggamma(i+2-1)-loggamma(i+2-2)))

20-element Array{Float64,1}:
  1.376764595972871
  0.6741204672279988
  0.3011797108310776
 -0.916927444098149
 -0.42236137845335125
 -0.5331713760596377
 -0.07585012860342231
 -0.36892141780946436
 -1.302999798139738
  1.4456279860955934
  0.18952276764035603
  0.4228330205899126
 -0.28912856965799705
  2.156092217339084
 -0.7256887694457368
  0.512434736753446
 -1.353369107156827
  0.1871853798611857
 -0.6777581208630928
  0.11246017030303562

In [56]:
sqrt(2)

1.4142135623730951

In [49]:
V

20×11 Array{Float64,2}:
 0.0  1.41421   1.37676    -3.73824  …  -1790.62    1147.69    32864.3
 0.0  1.41421   0.67412    -4.62069     -1068.92    5553.75    21094.6
 0.0  1.41421   0.30118    -4.84343      -500.258   6806.2     10029.3
 0.0  1.41421  -0.916927   -4.38412      1381.93    4286.58   -26792.3
 0.0  1.41421  -0.422361   -4.78974       693.815   6498.96   -13858.0
 0.0  1.41421  -0.533171   -4.7249   …    864.125   6132.46   -17181.2
 0.0  1.41421  -0.0758501  -4.89546       127.337   7107.14    -2561.98
 0.0  1.41421  -0.368921   -4.81563       609.279   6646.71   -12191.5
 0.0  1.41421  -1.303      -3.85929      1743.99    1700.98   -32365.5
 0.0  1.41421   1.44563    -3.61922     -1826.95     619.055   33153.2
 0.0  1.41421   0.189523   -4.87698  …   -316.97    6999.92     6369.33
 0.0  1.41421   0.422833   -4.78949      -694.555   6497.57    13872.6
 0.0  1.41421  -0.289129   -4.84779       480.671   6831.27    -9639.48
 0.0  1.41421   2.15609    -2.05222     -1755.24  

In [43]:
i = 4
exp(loggamma(i)-loggamma(i-1))

2.9999999999999996

In [46]:
factorial(3)/factorial(2)

3.0

In [21]:
FamilyPhyHermite[5].(randn(10))

10-element Array{Float64,1}:
   2.2247870592109162
 327.52893157772706
   5.811891508178892
 -21.687953327043342
 -20.43891961493114
  11.879144022764326
  -5.454203097009377
  35.07899495511365
 -23.029715915331195
 -19.576354543164022

### Rescale variables

In [24]:
### Rescale variable 
ens = EnsembleState(randn(100,1000))
x̄ = mean(ens)
S = cov(ens) + 20*I
C = cholesky(S)
@time inv(C.L)

UndefVarError: UndefVarError: cov not defined

In [25]:
?ldiv!

search: ldiv! splitdrive



```
ldiv!(Y, A, B) -> Y
```

Compute `A \ B` in-place and store the result in `Y`, returning the result.

The argument `A` should *not* be a matrix.  Rather, instead of matrices it should be a factorization object (e.g. produced by [`factorize`](@ref) or [`cholesky`](@ref)). The reason for this is that factorization itself is both expensive and typically allocates memory (although it can also be done in-place via, e.g., [`lu!`](@ref)), and performance-critical situations requiring `ldiv!` usually also require fine-grained control over the factorization of `A`.

# Examples

```jldoctest
julia> A = [1 2.2 4; 3.1 0.2 3; 4 1 2];

julia> X = [1; 2.5; 3];

julia> Y = zero(X);

julia> ldiv!(Y, qr(A), X);

julia> Y
3-element Array{Float64,1}:
  0.7128099173553719
 -0.051652892561983674
  0.10020661157024757

julia> A\X
3-element Array{Float64,1}:
  0.7128099173553719
 -0.05165289256198333
  0.10020661157024785
```

---

```
ldiv!(A, B)
```

Compute `A \ B` in-place and overwriting `B` to store the result.

The argument `A` should *not* be a matrix.  Rather, instead of matrices it should be a factorization object (e.g. produced by [`factorize`](@ref) or [`cholesky`](@ref)). The reason for this is that factorization itself is both expensive and typically allocates memory (although it can also be done in-place via, e.g., [`lu!`](@ref)), and performance-critical situations requiring `ldiv!` usually also require fine-grained control over the factorization of `A`.

# Examples

```jldoctest
julia> A = [1 2.2 4; 3.1 0.2 3; 4 1 2];

julia> X = [1; 2.5; 3];

julia> Y = copy(X);

julia> ldiv!(qr(A), X);

julia> X
3-element Array{Float64,1}:
  0.7128099173553719
 -0.051652892561983674
  0.10020661157024757

julia> A\Y
3-element Array{Float64,1}:
  0.7128099173553719
 -0.05165289256198333
  0.10020661157024785
```

---

```
ldiv!(a::Number, B::AbstractArray)
```

Divide each entry in an array `B` by a scalar `a` overwriting `B` in-place.  Use [`rdiv!`](@ref) to divide scalar from right.

# Examples

```jldoctest
julia> B = [1.0 2.0; 3.0 4.0]
2×2 Array{Float64,2}:
 1.0  2.0
 3.0  4.0

julia> ldiv!(2.0, B)
2×2 Array{Float64,2}:
 0.5  1.0
 1.5  2.0
```

---

Solve Hy = rhs for a non-square Hessenberg matrix. Note that `H` is also modified as is it converted to an upper triangular matrix via Given's rotations


In [26]:
?cholesky

search: cholesky cholesky! Cholesky CholeskyPivoted



```
cholesky(A, Val(false); check = true) -> Cholesky
```

Compute the Cholesky factorization of a dense symmetric positive definite matrix `A` and return a `Cholesky` factorization. The matrix `A` can either be a [`Symmetric`](@ref) or [`Hermitian`](@ref) `StridedMatrix` or a *perfectly* symmetric or Hermitian `StridedMatrix`. The triangular Cholesky factor can be obtained from the factorization `F` with: `F.L` and `F.U`. The following functions are available for `Cholesky` objects: [`size`](@ref), [`\`](@ref), [`inv`](@ref), [`det`](@ref), [`logdet`](@ref) and [`isposdef`](@ref).

When `check = true`, an error is thrown if the decomposition fails. When `check = false`, responsibility for checking the decomposition's validity (via [`issuccess`](@ref)) lies with the user.

# Examples

```jldoctest
julia> A = [4. 12. -16.; 12. 37. -43.; -16. -43. 98.]
3×3 Array{Float64,2}:
   4.0   12.0  -16.0
  12.0   37.0  -43.0
 -16.0  -43.0   98.0

julia> C = cholesky(A)
Cholesky{Float64,Array{Float64,2}}
U factor:
3×3 UpperTriangular{Float64,Array{Float64,2}}:
 2.0  6.0  -8.0
  ⋅   1.0   5.0
  ⋅    ⋅    3.0

julia> C.U
3×3 UpperTriangular{Float64,Array{Float64,2}}:
 2.0  6.0  -8.0
  ⋅   1.0   5.0
  ⋅    ⋅    3.0

julia> C.L
3×3 LowerTriangular{Float64,Array{Float64,2}}:
  2.0   ⋅    ⋅
  6.0  1.0   ⋅
 -8.0  5.0  3.0

julia> C.L * C.U == A
true
```

---

```
cholesky(A, Val(true); tol = 0.0, check = true) -> CholeskyPivoted
```

Compute the pivoted Cholesky factorization of a dense symmetric positive semi-definite matrix `A` and return a `CholeskyPivoted` factorization. The matrix `A` can either be a [`Symmetric`](@ref) or [`Hermitian`](@ref) `StridedMatrix` or a *perfectly* symmetric or Hermitian `StridedMatrix`. The triangular Cholesky factor can be obtained from the factorization `F` with: `F.L` and `F.U`. The following functions are available for `CholeskyPivoted` objects: [`size`](@ref), [`\`](@ref), [`inv`](@ref), [`det`](@ref), and [`rank`](@ref). The argument `tol` determines the tolerance for determining the rank. For negative values, the tolerance is the machine precision.

When `check = true`, an error is thrown if the decomposition fails. When `check = false`, responsibility for checking the decomposition's validity (via [`issuccess`](@ref)) lies with the user.

---

```
cholesky(A; shift = 0.0, check = true, perm = nothing) -> CHOLMOD.Factor
```

Compute the Cholesky factorization of a sparse positive definite matrix `A`. `A` must be a [`SparseMatrixCSC`](@ref) or a [`Symmetric`](@ref)/[`Hermitian`](@ref) view of a `SparseMatrixCSC`. Note that even if `A` doesn't have the type tag, it must still be symmetric or Hermitian. If `perm` is not given, a fill-reducing permutation is used. `F = cholesky(A)` is most frequently used to solve systems of equations with `F\b`, but also the methods [`diag`](@ref), [`det`](@ref), and [`logdet`](@ref) are defined for `F`. You can also extract individual factors from `F`, using `F.L`. However, since pivoting is on by default, the factorization is internally represented as `A == P'*L*L'*P` with a permutation matrix `P`; using just `L` without accounting for `P` will give incorrect answers. To include the effects of permutation, it's typically preferable to extract "combined" factors like `PtL = F.PtL` (the equivalent of `P'*L`) and `LtP = F.UP` (the equivalent of `L'*P`).

When `check = true`, an error is thrown if the decomposition fails. When `check = false`, responsibility for checking the decomposition's validity (via [`issuccess`](@ref)) lies with the user.

Setting the optional `shift` keyword argument computes the factorization of `A+shift*I` instead of `A`. If the `perm` argument is provided, it should be a permutation of `1:size(A,1)` giving the ordering to use (instead of CHOLMOD's default AMD ordering).

# Examples

In the following example, the fill-reducing permutation used is `[3, 2, 1]`. If `perm` is set to `1:3` to enforce no permutation, the number of nonzero elements in the factor is 6.

```jldoctest
julia> A = [2 1 1; 1 2 0; 1 0 2]
3×3 Array{Int64,2}:
 2  1  1
 1  2  0
 1  0  2

julia> C = cholesky(sparse(A))
SuiteSparse.CHOLMOD.Factor{Float64}
type:    LLt
method:  simplicial
maxnnz:  5
nnz:     5
success: true

julia> C.p
3-element Array{Int64,1}:
 3
 2
 1

julia> L = sparse(C.L);

julia> Matrix(L)
3×3 Array{Float64,2}:
 1.41421   0.0       0.0
 0.0       1.41421   0.0
 0.707107  0.707107  1.0

julia> L * L' ≈ A[C.p, C.p]
true

julia> P = sparse(1:3, C.p, ones(3))
3×3 SparseMatrixCSC{Float64,Int64} with 3 stored entries:
  [3, 1]  =  1.0
  [2, 2]  =  1.0
  [1, 3]  =  1.0

julia> P' * L * L' * P ≈ A
true

julia> C = cholesky(sparse(A), perm=1:3)
SuiteSparse.CHOLMOD.Factor{Float64}
type:    LLt
method:  simplicial
maxnnz:  6
nnz:     6
success: true

julia> L = sparse(C.L);

julia> Matrix(L)
3×3 Array{Float64,2}:
 1.41421    0.0       0.0
 0.707107   1.22474   0.0
 0.707107  -0.408248  1.1547

julia> L * L' ≈ A
true
```

!!! note
    This method uses the CHOLMOD library from SuiteSparse, which only supports doubles or complex doubles. Input matrices not of those element types will be converted to `SparseMatrixCSC{Float64}` or `SparseMatrixCSC{ComplexF64}` as appropriate.

    Many other functions from CHOLMOD are wrapped but not exported from the `Base.SparseArrays.CHOLMOD` module.

